# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import json
sys.path.append('..')

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv('../WeatherPy/Output/city_weather.csv')
city_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kentau,43.5206,68.5094,38.32,70,100,3.31,KZ,1612492642
1,1,severo-kurilsk,50.6789,156.1250,12.99,85,73,10.04,RU,1612492643
2,2,ribeira grande,38.5167,-28.7000,55.22,69,38,19.64,PT,1612492644
3,3,shenzhen,22.5455,114.0683,72.00,65,17,5.01,CN,1612492598
4,4,barrow,71.2906,-156.7887,-27.40,75,75,8.05,US,1612492676
...,...,...,...,...,...,...,...,...,...,...
475,475,paamiut,61.9940,-49.6678,17.65,74,0,5.59,GL,1612493099
476,476,montepuez,-13.1256,38.9997,68.05,97,10,1.21,MZ,1612493252
477,477,oriximina,-1.7656,-55.8661,77.23,85,18,4.97,BR,1612493253
478,478,tyukhtet,56.5389,89.3011,3.79,94,96,6.87,RU,1612493254


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
fig = gmaps.figure()
locations = city_weather_df[['Lat', 'Lng']]
heat_layer = gmaps.heatmap_layer(locations, weights = city_weather_df['Humidity'], dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_city_weather_df = city_weather_df.loc[(city_weather_df['Max Temp'] > 70) & (city_weather_df['Max Temp'] < 80) 
                            & (city_weather_df['Wind Speed'] <= 10) & (city_weather_df['Cloudiness'] == 0), :]

ideal_city_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,lazaro cardenas,17.9583,-102.2000,73.71,81,0,3.27,MX,1612492750
109,109,boma,7.0805,-2.1697,72.86,92,0,2.82,GH,1612492807
206,206,kon tum,14.3500,108.0000,78.58,48,0,4.94,VN,1612492908
220,220,comodoro rivadavia,-45.8667,-67.5000,73.40,56,0,3.44,AR,1612492935
241,241,general roca,-39.0333,-67.5833,75.20,57,0,2.30,AR,1612492960
243,243,caravelas,-17.7125,-39.2481,77.00,83,0,8.01,BR,1612492963
272,272,tura,25.5198,90.2201,71.60,43,0,0.87,IN,1612492820
363,363,chatakonda,17.5500,80.6500,70.21,57,0,3.42,IN,1612493110
391,391,manacapuru,-3.2997,-60.6206,77.00,94,0,2.66,BR,1612493145
426,426,obuasi,6.2060,-1.6619,73.62,94,0,2.64,GH,1612493190


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_city_weather_df
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,lazaro cardenas,17.9583,-102.2000,73.71,81,0,3.27,MX,1612492750
109,109,boma,7.0805,-2.1697,72.86,92,0,2.82,GH,1612492807
206,206,kon tum,14.3500,108.0000,78.58,48,0,4.94,VN,1612492908
220,220,comodoro rivadavia,-45.8667,-67.5000,73.40,56,0,3.44,AR,1612492935
241,241,general roca,-39.0333,-67.5833,75.20,57,0,2.30,AR,1612492960
243,243,caravelas,-17.7125,-39.2481,77.00,83,0,8.01,BR,1612492963
272,272,tura,25.5198,90.2201,71.60,43,0,0.87,IN,1612492820
363,363,chatakonda,17.5500,80.6500,70.21,57,0,3.42,IN,1612493110
391,391,manacapuru,-3.2997,-60.6206,77.00,94,0,2.66,BR,1612493145
426,426,obuasi,6.2060,-1.6619,73.62,94,0,2.64,GH,1612493190


In [6]:
hotel_names = []
geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
for index, row in hotel_df.iterrows():
    params = {
        'address': row['City']
        , 'key': g_key
    }
    city_response = requests.get(geocode_url, params).json()
    
    lat = city_response['results'][0]['geometry']['location']['lat']
    lng = city_response['results'][0]['geometry']['location']['lng']
    location = f'{lat},{lng}'
    keyword = '&keyword=hotel'
    nearby_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    final_url = f'{nearby_url}&location={location}{keyword}&rankby=distance&key={g_key}'
    hotel = requests.get(final_url).json()
#     print(hotel['results'][0]['name'])
    try:
        hotel_names.append(hotel['results'][0]['name'])
    except:
        hotel_names.append('')
hotel_df['Hotel Name'] = hotel_names
hotel_df

<ipython-input-6-a3e35e3569c4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_names


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,63,lazaro cardenas,17.9583,-102.2000,73.71,81,0,3.27,MX,1612492750,Sleep Inn
109,109,boma,7.0805,-2.1697,72.86,92,0,2.82,GH,1612492807,Renaissance Milwaukee West
206,206,kon tum,14.3500,108.0000,78.58,48,0,4.94,VN,1612492908,Khách sạn Bích Phương kontum
220,220,comodoro rivadavia,-45.8667,-67.5000,73.40,56,0,3.44,AR,1612492935,Hotel Victoria
241,241,general roca,-39.0333,-67.5833,75.20,57,0,2.30,AR,1612492960,Rincon Del Valle
243,243,caravelas,-17.7125,-39.2481,77.00,83,0,8.01,BR,1612492963,Pousada Liberdade
272,272,tura,25.5198,90.2201,71.60,43,0,0.87,IN,1612492820,Fort Brady Hotel
363,363,chatakonda,17.5500,80.6500,70.21,57,0,3.42,IN,1612493110,Hotel Sri Balaji A/c
391,391,manacapuru,-3.2997,-60.6206,77.00,94,0,2.66,BR,1612493145,OYO TBS Hotel
426,426,obuasi,6.2060,-1.6619,73.62,94,0,2.64,GH,1612493190,Coconut Grove Miners Lodge


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))